# Import geo-data of "ÖV" in Geneva using swisstopo API

## Libraries and settings

In [1]:
# Libraries
import os
import requests
import json
import folium
from pandas import json_normalize

# Ignore warnings
import warnings
warnings.filterwarnings("ignore")

# Show current working directory
print(os.getcwd())

/workspaces/project_arm_airbnb/data_oev


## API request

### Train stations

In [28]:
# Overpass API URL
url = 'http://overpass-api.de/api/interpreter'

# Overpass turbo query
query = f"""
        [out:json];
        area["ISO3166-2"="CH-GE"][admin_level=4];
        node ["railway"="station"](area);
        out;"""

# Web API request
r = requests.get(url, params={'data': query})
data = r.json()['elements']

# Save data to file
with open('station.json', 'w') as json_file:
    json.dump(data, json_file)

# Store data in data frame
df_train = json_normalize(data)

# Number of rows and columns
print(df_train.shape)

# First rows
df_train.head()

(13, 38)


,type,id,lat,lon,tags.ele,tags.internet_access,tags.internet_access:fee,tags.internet_access:operator,tags.name,tags.name:de,...,tags.name:azb,tags.name:en,tags.name:fa,tags.name:kk-Arab,tags.name:pnb,tags.name:sd,tags.name:ur,tags.network,tags.network:short,tags.network:wikidata
0,node,2388447150,46.231334,6.110271,424,wlan,no,SBB,Genève-Aéroport,Genf Flughafen,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,node,2754246079,46.220426,6.094808,428,NaN,NaN,NaN,Vernier,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,node,3053167578,46.263684,6.161146,386,NaN,NaN,NaN,Creux-de-Genthod,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,node,3053167580,46.279740,6.165805,388,no,NaN,NaN,Versoix,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,node,3057742963,46.222273,6.076288,434,NaN,NaN,NaN,Meyrin,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [31]:
# load data from df_train intro locations_train
locations_train = df_train[['lat','lon', "tags.name"]]

#Create map
m = folium.Map(location=[46.2044, 6.1432], zoom_start=12)

# add marker to sympols
for i in range(0, len(locations_train)):
    folium.Marker([locations_train.iloc[i]['lat'], locations_train.iloc[i]['lon']],
                  popup=locations_train.iloc[i]['tags.name']).add_to(m)
    
# plot map
m


In [30]:
# Load data
locations_train = df_train[["lat", "lon", "tags.public_transport", "tags.name"]].loc[df_train["tags.public_transport"].isin(['tags.name'])]
print(locations_train.head(5))

# Number of rows and columns
print(locations_train.shape)

# Create map
map = folium.Map(location=[locations_train.lat.mean(), 
                           locations_train.lon.mean()], 
                 zoom_start=8, 
                 control_scale=True)

# Add maker symbols
for index, location_info in locations_train.iterrows():
    folium.Marker([location_info["lat"], 
                   location_info["lon"]], 
                  popup=location_info["tags.name"]).add_to(map)

# Plot map
map

Empty DataFrame
Columns: [lat, lon, tags.public_transport, tags.name]
Index: []
(0, 4)


ValueError: Location values cannot contain NaNs.

### Tram stops

In [9]:
# Overpass turbo query
query = f"""
        [out:json];
        area["ISO3166-2"="CH-GE"][admin_level=4];
        node ["railway"="tram_stop"](area);
        out;"""

# Web API request
r = requests.get(url, params={'data': query})
data = r.json()['elements']

# Save data to file
with open('tram.json', 'w') as json_file:
    json.dump(data, json_file)

# Store data in data frame
df_tram = json_normalize(data)

# Number of rows and columns
print(df_tram.shape)

# First rows
df_tram.head()


(180, 29)


,type,id,lat,lon,tags.ele,tags.name,tags.network,tags.operator,tags.public_transport,tags.railway,...,tags.website,tags.url,tags.contact:website,tags.layer,tags.opening_hours,tags.addr:city,tags.addr:postcode,tags.addr:street,tags.note,tags.start_date
0,node,35416998,46.218462,6.110622,430,Balexert,18,TPG,stop_position,tram_stop,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,node,35420048,46.221020,6.141444,401,Collège Sismondi,NaN,TPG,stop_position,tram_stop,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,node,36663771,46.188926,6.131024,378,Pictet-Thellusson,NaN,TPG,stop_position,tram_stop,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,node,297910640,46.200564,6.140755,375,Cirque,NaN,TPG,stop_position,tram_stop,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,node,297914745,46.200847,6.143642,382,Place de Neuve,Unireso,TPG,stop_position,tram_stop,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [15]:
# Load data
locations_tram = df_tram[["lat", "lon", "tags.railway", "tags.name"]].loc[df_tram["tags.railway"].isin(['tram_stop'])]
print(locations_tram.head(5))

# Number of rows and columns
print(locations_tram.shape)

# Create map
map = folium.Map(location=[locations_tram.lat.mean(), 
                           locations_tram.lon.mean()], 
                 zoom_start=8, 
                 control_scale=True)

# Add maker symbols
for index, location_info in locations_tram.iterrows():
    folium.Marker([location_info["lat"], 
                   location_info["lon"]], 
                  popup=location_info["tags.name"]).add_to(map)

# Plot map
map

         lat       lon tags.railway          tags.name
0  46.218462  6.110622    tram_stop           Balexert
1  46.221020  6.141444    tram_stop   Collège Sismondi
2  46.188926  6.131024    tram_stop  Pictet-Thellusson
3  46.200564  6.140755    tram_stop             Cirque
4  46.200847  6.143642    tram_stop     Place de Neuve
(180, 4)


In [16]:
# drop duplicates based on tags.name
locations_tram = locations_tram.drop_duplicates(subset="tags.name")
print(locations_tram.head(5))

# Number of rows and columns
print(locations_tram.shape)

# Create map
map = folium.Map(location=[locations_tram.lat.mean(), 
                           locations_tram.lon.mean()], 
                 zoom_start=8, 
                 control_scale=True)

# Add maker symbols
for index, location_info in locations_tram.iterrows():
    folium.Marker([location_info["lat"], 
                   location_info["lon"]], 
                  popup=location_info["tags.name"]).add_to(map)

# Plot map
map

         lat       lon tags.railway          tags.name
0  46.218462  6.110622    tram_stop           Balexert
1  46.221020  6.141444    tram_stop   Collège Sismondi
2  46.188926  6.131024    tram_stop  Pictet-Thellusson
3  46.200564  6.140755    tram_stop             Cirque
4  46.200847  6.143642    tram_stop     Place de Neuve
(82, 4)


### Bus stops

In [12]:
# Overpass turbo query
query = f"""
        [out:json];
        area["ISO3166-2"="CH-GE"][admin_level=4];
        node ["highway"="bus_stop"](area);
        out;"""

# Web API request
r = requests.get(url, params={'data': query})
data = r.json()['elements']

# Save data to file
with open('bus.json', 'w') as json_file:
    json.dump(data, json_file)

# Store data in data frame
df_bus = json_normalize(data)

# Number of rows and columns
print(df_bus.shape)

# First rows
df_bus.head()

(1451, 54)


,type,id,lat,lon,tags.bus,tags.ele,tags.highway,tags.name,tags.operator,tags.public_transport,...,tags.contact:website,tags.name:zh,tags.nat_name,tags.description,tags.was:highway,tags.note,tags.website,tags.not:network:wikidata,tags.start_date,tags.tourist_bus
0,node,497307,46.222755,6.139638,yes,372,bus_stop,Nations,TPG,stop_position,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,node,35419951,46.220048,6.145630,yes,390,bus_stop,Maison de la Paix,TPG,stop_position,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,node,36473383,46.156441,6.003172,yes,422,bus_stop,Athenaz-Passeiry,TPG,stop_position,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,node,36473725,46.144418,5.965679,yes,341,bus_stop,Chancy-Douane,TPG,stop_position,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,node,36647210,46.220019,6.085083,yes,NaN,bus_stop,Signal,TPG,stop_position,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [1]:
# Load data
locations = df_bus[["lat", "lon", "tags.highway", "tags.name"]].loc[df_bus["tags.highway"].isin(['bus_stop'])]
print(locations.head(5))

# Number of rows and columns
print(locations.shape)

# Create map
map = folium.Map(location=[locations.lat.mean(), 
                           locations.lon.mean()], 
                 zoom_start=8, 
                 control_scale=True)

# Add maker symbols
for index, location_info in locations.iterrows():
    folium.Marker([location_info["lat"], 
                   location_info["lon"]], 
                  popup=location_info["tags.name"]).add_to(map)

# Plot map
map

NameError: name 'df_bus' is not defined

In [6]:
# drop duplicates based on tags.name
locations = locations.drop_duplicates(subset="tags.name")
print(locations.head(5))

# Number of rows and columns
print(locations.shape)

# Create map
map = folium.Map(location=[locations.lat.mean(), 
                           locations.lon.mean()], 
                 zoom_start=8, 
                 control_scale=True)

# Add maker symbols
for index, location_info in locations.iterrows():
    folium.Marker([location_info["lat"], 
                   location_info["lon"]], 
                  popup=location_info["tags.name"]).add_to(map)

# Plot map
map

         lat       lon tags.highway          tags.name
0  46.222755  6.139638     bus_stop            Nations
1  46.220048  6.145630     bus_stop  Maison de la Paix
2  46.156441  6.003172     bus_stop   Athenaz-Passeiry
3  46.144418  5.965679     bus_stop      Chancy-Douane
4  46.220019  6.085083     bus_stop             Signal
(738, 4)
